Import libraries

In [1]:
import pandas as pd
import numpy as np # library to handle data in a vectorized manner

import geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
! pip install --upgrade pip --user

Requirement already up-to-date: pip in /home/sedna/.local/lib/python3.7/site-packages (19.3.1)


In [3]:
neighbourhoods=pd.read_csv('/home/sedna/Downloads/PROJECT-COURSERA-CAPSTONE/DATA/neighbourhoods_london.csv',index_col=0)

In [4]:
neighbourhoods.index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            554, 555, 556, 557, 558, 559, 560, 561, 562, 563],
           dtype='int64', length=559)

In [5]:
p=len(neighbourhoods.index)

In [7]:


address = 'Southbank, London,UK'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate are {}, {}.'.format(latitude, longitude))

The geograpical coordinate are 51.4287749, 0.0696267.


In [8]:
address = 'Waterloo, London,UK'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate are {}, {}.'.format(latitude, longitude))

The geograpical coordinate are 51.5025314, -0.1125511.


In [9]:
address = 'Kings Cross, London,UK'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate are {}, {}.'.format(latitude, longitude))

The geograpical coordinate are 51.5320529, -0.1206259.


In [10]:
address = 'Bermondsey, London,UK'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate are {}, {}.'.format(latitude, longitude))

The geograpical coordinate are 51.4970125, -0.063268.


In [ ]:
neighbourhoods.loc[169,'Latitude']

Let's explore one of the most visited neighbourhoods in London: Osterley


In [ ]:
neighbourhoods.loc[neighbourhoods.Neighbourhood=="Creekmouth"]

In [ ]:


neighbourhood_latitude = neighbourhoods.loc[347, 'Latitude'] # neighborhood latitude value 
neighbourhood_longitude = neighbourhoods.loc[380, 'Longitude'] # neighborhood longitude value

neighbourhood_name =neighbourhoods.loc[347, 'Neighbourhood'] # neighborhood name


In [ ]:
print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, neighbourhood_latitude, neighbourhood_longitude))


In [ ]:
CLIENT_ID = 'X3INNHZFS0RE4YAMNP5CCG4UVUP4S5K1S4NZIIXKHN2UNXJW' # your Foursquare ID
CLIENT_SECRET = '20EOF0IP0MALLD0RO0MM1QOCLEHXC35L4U3JHMETTEF24YJR' # your Foursquare Secret

VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Now, let's get the top 100 venues that are in Orteley a radius of 500 meters. First, let's create the GET request URL. Name your URL url.


In [ ]:
latitude=neighbourhood_latitude 
longitude=neighbourhood_longitude
radius=500 
LIMIT=100
url='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url


Send the GET request and examine the resutls


In [ ]:
results = requests.get(url).json() 
results

import json
with open('results.json', 'r') as f:
    results=json.load(f)

Using this function from Foursquare

In [ ]:
#function that extracts the category of the venue
def get_category_type(row): 
        try:
            categories_list = row['categories'] 
        except: 
            categories_list = row['venue.categories']

        if len(categories_list) == 0:
            return None
        else:
            return categories_list[0]['name']
def get_category_primary(file,col):
        Category_list=[]
        for ii in range(file.shape[0]): 
              Category_list.append(file[col][ii][0]['icon']['prefix'].split('/')[5])
        return Category_list
          

In [ ]:
#Clean json and structure it into a pandas dataframe
try:
    
    venues = results['response']['groups'][0]['items']
except:
    venues = results['response']['items']

In [ ]:
venues

In [ ]:
venues_list=[] 
# return only relevant information for each nearby venue
venues_list.append([(
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            v['venue']['categories'][0]['icon']['prefix'].split('/')[5]) for v in venues])
    

In [ ]:
venues_list

In [ ]:

  
url=venues[0]['venue']['categories'][0]['icon']['prefix'].split('/')[5]
#['icon']['prefix'].split('/')[5]

nearby_venues = json_normalize(venues) # flatten JSON

#print(nearby_venues.shape[0])

#print(nearby_venues['venue.categories'][0][0]['icon']['prefix'].split('/')[5])
Categories =get_category_primary(nearby_venues,'venue.categories')

#filter columns
filtered_columns = ['venue.name', 'venue.categories','venue.location.lat', 'venue.location.lng'] 

nearby_venues =nearby_venues.loc[:, filtered_columns]

#filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
#nearby_venues['venue.Categories'] = Categories
#clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.insert(2,'Categories', Categories)
nearby_venues

nearby_venues



In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))